# Creating a UDF for the YoloV7 Model

This adapts the object detection script in https://github.com/RizwanMunawar/yolov7-object-tracking for Pixeltable.

This script needs to be run in the environment created for that repo. The UDF is stored in a database `functions` and can subsequently be used in queries and computed columns without having access to the modules used in the script.

In [ ]:
import torchvision

import models, utils, PIL, thop
from models.experimental import attempt_load
from utils.torch_utils import select_device
from utils.general import non_max_suppression
%load_ext autoreload
%autoreload 2

In [ ]:
weights_file_name = 'yolov7.pt'
device = select_device('')

model = attempt_load(weights_file_name, map_location=device)  # load FP32 model

**`detect()` takes a PIL.Image and returns a numpy array of detections**

Each detection is a numpy array of 6 floats, containing
* the bounding box (as xyxy)
* the confidence
* the class

In [ ]:
def detect(img):
    expected_img_size = 640  # required by model
    # resize img to expected size while maintaining aspect ratio
    resize_ratio = min(expected_img_size / img.width, expected_img_size / img.height)
    new_width = int(resize_ratio * img.width)
    new_height = int(resize_ratio * img.height)
    img = PIL.ImageOps.pad(img, (expected_img_size, expected_img_size), color=(114, 114, 114))

    img_tensor = torchvision.transforms.functional.pil_to_tensor(img) / 255.0
    # turn into batch of 1
    img_tensor = img_tensor.unsqueeze(0)

    model_output = model(img_tensor)
    pred = model_output[0]
    pred = non_max_suppression(pred)
    detections = pred[0].numpy(force=True)
    detections[:, :4] /= resize_ratio  # convert to original image size
    return detections

Sanity check

In [ ]:
img_file = '/home/marcel/pixeltable/pixeltable/tests/data/imagenette2-160/n03445777_2563.JPEG'
img = PIL.Image.open(img_file)

print(img.size)
display(img)

In [ ]:
detect(img)

We create database `functions` (or get handle to it)

In [ ]:
import sys
sys.path.append('/home/marcel/pixeltable')

import pixeltable as pt
from pixeltable.function import Function, FunctionRegistry
from pixeltable.type_system import ArrayType, ImageType, ColumnType, JsonType

cl = pt.Client()
try:
    db = cl.get_db('functions')
except pt.UnknownEntityError:
    db = cl.create_db('functions')

We then store `detect()` as a named UDF 'yolov7' in the db

In [ ]:
FunctionRegistry.register_pickled_module(models)
FunctionRegistry.register_pickled_module(utils)
FunctionRegistry.register_pickled_module(thop)

In [ ]:
yolov7_udf = Function(ArrayType((None, 6), dtype=ColumnType.Type.FLOAT), [ImageType()], eval_fn=detect)
db.drop_function('yolov7')
db.create_function('yolov7', yolov7_udf)

Let's verify that it worked.

We're starting with a fresh client to make sure we're not simply referencing cached data.

In [ ]:
cl = pt.Client()
db = cl.get_db('functions')
yolov7_udf = db.load_function('yolov7')

In [ ]:
detections = yolov7_udf.eval_fn(img)
detections